In [10]:
# At the top of GitHub.ipynb
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI

load_dotenv()

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

GPT_MODEL = ChatOpenAI(model="gpt-4o", temperature=0.2)

ModuleNotFoundError: No module named 'langchain'

In [11]:
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import Runnable
from langchain.chat_models import ChatOpenAI
import os
import requests
import json

ModuleNotFoundError: No module named 'langchain'

In [12]:
# === NODE: Generate Repo Name ===

def generate_repo_name(state):
    files = state.get("files", [])
    file_names = [f["name"] for f in files]
    prompt = "Suggest a GitHub repository name based on the following project files:\n" + "\n".join(file_names)
    response = GPT_MODEL.invoke([HumanMessage(content=prompt)])
    repo_name = response.content.strip().replace(" ", "-").lower()
    return {"repo_name": repo_name}

In [ ]:
# === NODE: Generate README ===

def generate_readme(state):
    files = state.get("files", [])
    filenames = [f["name"] for f in files]
    prompt = f"Write a professional README.md file for a project with these files:\n{filenames}"
    response = GPT_MODEL.invoke([HumanMessage(content=prompt)])
    return {"readme": response.content}

In [ ]:
# === NODE: Summarize Files ===

def summarize_files(state):
    files = state.get("files", [])
    summaries = {}
    for file in files:
        name = file["name"]
        content = file["content"]
        prompt = f"Summarize the following code file named '{name}':\n\n{content}"
        summary = GPT_MODEL.invoke([HumanMessage(content=prompt)]).content
        summaries[name] = summary
    return {"summaries": summaries}

In [ ]:
# === NODE: Create GitHub Repo and Upload ===

def create_and_upload_repo(state):
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    repo_name = state["repo_name"]
    readme = state["readme"]
    summaries = state["summaries"]
    files = state["files"]

    # Step 1: Create repo
    repo_data = {"name": repo_name, "auto_init": False, "private": False}
    repo_resp = requests.post("https://api.github.com/user/repos", headers=headers, json=repo_data)
    if repo_resp.status_code != 201:
        raise Exception(f"GitHub repo creation failed: {repo_resp.json()}")

    # Step 2: Upload README
    upload_file_to_github(repo_name, "README.md", readme, headers)

    # Step 3: Upload other files
    for file in files:
        upload_file_to_github(repo_name, file["name"], file["content"], headers)

    # Step 4: Upload summary file
    summary_content = json.dumps(summaries, indent=2)
    upload_file_to_github(repo_name, "code_summaries.json", summary_content, headers)

    return {"github_repo": f"https://github.com/your_username/{repo_name}"}

In [ ]:
# === Helper: Upload File ===

def upload_file_to_github(repo_name, path, content, headers):
    from base64 import b64encode
    url = f"https://api.github.com/repos/your_username/{repo_name}/contents/{path}"
    data = {
        "message": f"Add {path}",
        "content": b64encode(content.encode()).decode()
    }
    r = requests.put(url, headers=headers, json=data)
    if r.status_code not in (200, 201):
        raise Exception(f"Failed to upload {path}: {r.text}")

In [ ]:
# === STATE & GRAPH ===

class RepoState(dict):
    pass

graph = StateGraph(RepoState)
graph.add_node("generate_repo_name", generate_repo_name)
graph.add_node("generate_readme", generate_readme)
graph.add_node("summarize_files", summarize_files)
graph.add_node("upload_repo", create_and_upload_repo)

graph.set_entry_point("generate_repo_name")
graph.add_edge("generate_repo_name", "generate_readme")
graph.add_edge("generate_readme", "summarize_files")
graph.add_edge("summarize_files", "upload_repo")
graph.add_edge("upload_repo", END)

final_graph = graph.compile()